## HYPERPARAMETER TUNING WITH RAY TUNE

In [1]:
from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

In [2]:
def load_data(data_dir="./"):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True, download=True, transform=transform)

    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False, download=True, transform=transform)

    return trainset, testset

In [3]:
class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
def train_cifar(config, checkpoint_dir=None, data_dir=None):
    net = Net(config["l1"], config["l2"])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader = torch.utils.data.DataLoader(
        train_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)

    for epoch in range(10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((net.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
    print("Finished Training")

In [5]:
def test_accuracy(net, device="cpu"):
    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [6]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    data_dir = os.path.abspath("./data")
    load_data(data_dir)
    config = {
        "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16])
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        partial(train_cifar, data_dir=data_dir),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    best_trained_model = Net(best_trial.config["l1"], best_trial.config["l2"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_acc = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {}".format(test_acc))


if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=10, max_num_epochs=10, gpus_per_trial=0)

26.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

37.2%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

55.2%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

72.1%IOPub

Extracting C:\my_code\Computer_Vision\Torch_Tutorial\Optimization\data\cifar-10-python.tar.gz to C:\my_code\Computer_Vision\Torch_Tutorial\Optimization\data
Files already downloaded and verified


2021-07-28 00:39:33,934	INFO services.py:1274 -- View the Ray dashboard at http://127.0.0.1:8265
2021-07-28 00:39:46,672	WARNING experiment.py:294 -- No name detected on trainable. Using DEFAULT.
2021-07-28 00:39:46,673	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.
2021-07-28 00:39:46,988	ERROR syncer.py:72 -- Log sync requires rsync to be installed.


== Status ==
Memory usage on this node: 6.3/15.9 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 0/12 CPUs, 0/1 GPUs, 0.0/6.24 GiB heap, 0.0/3.12 GiB objects
Result logdir: C:\Users\PARK JIHO\ray_results\DEFAULT_2021-07-28_00-39-46
Number of trials: 10/10 (10 PENDING)
+---------------------+----------+-------+--------------+------+------+-------------+
| Trial name          | status   | loc   |   batch_size |   l1 |   l2 |          lr |
|---------------------+----------+-------+--------------+------+------+-------------|
| DEFAULT_df0ef_00000 | PENDING  |       |            8 |  128 |   16 | 0.00832281  |
| DEFAULT_df0ef_00001 | PENDING  |       |           16 |   64 |   64 | 0.038013    |
| DEFAULT_df0ef_00002 | PENDING  |       |            4 |   16 |    4 | 0.0278657   |
| DEFAULT_df0ef_00003 | PENDING  |       |            4 |    8 |    4 | 0.000381076 |
| DEFAULT_df0ef_00004 | PENDING  

(pid=12976) Traceback (most recent call last):
(pid=12976)   File "<string>", line 1, in <module>
(pid=12976)   File "C:\Users\PARK JIHO\anaconda3\envs\torch\lib\multiprocessing\spawn.py", line 105, in spawn_main
(pid=12976)     exitcode = _main(fd)
(pid=12976)   File "C:\Users\PARK JIHO\anaconda3\envs\torch\lib\multiprocessing\spawn.py", line 115, in _main
(pid=12976)     self = reduction.pickle.load(from_parent)
(pid=12976)   File "C:\Users\PARK JIHO\anaconda3\envs\torch\lib\site-packages\torch\__init__.py", line 117, in <module>
(pid=12976)     raise err
(pid=12976) OSError: [WinError 1455] �� �۾��� �Ϸ��ϱ� ���� ����¡ ������ �ʹ� �۽��ϴ�. Error loading "C:\Users\PARK JIHO\anaconda3\envs\torch\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.
(pid=13480) Traceback (most recent call last):
(pid=13480)   File "<string>", line 1, in <module>
(pid=13480)   File "C:\Users\PARK JIHO\anaconda3\envs\torch\lib\multiprocessing\spawn.py", line 105, in spawn_main

...

(pid=22428) [1,  2000] loss: 2.317
Result for DEFAULT_df0ef_00009:
  accuracy: 0.1243
  date: 2021-07-28_00-41-02
  done: false
  experiment_id: 1c6056f30c4843f092492f6cb727925e
  hostname: DESKTOP-G26MRLK
  iterations_since_restore: 1
  loss: 2.302574312210083
  node_ip: 192.168.219.101
  pid: 22428
  should_checkpoint: true
  time_since_restore: 41.067832946777344
  time_this_iter_s: 41.067832946777344
  time_total_s: 41.067832946777344
  timestamp: 1627400462
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: df0ef_00009
  
== Status ==
Memory usage on this node: 5.1/15.9 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: -2.302574312210083
Resources requested: 2.0/12 CPUs, 0/1 GPUs, 0.0/6.24 GiB heap, 0.0/3.12 GiB objects
Result logdir: C:\Users\PARK JIHO\ray_results\DEFAULT_2021-07-28_00-39-46
Number of trials: 10/10 (9 ERROR, 1 RUNNING)
+---------------------+----------+-----------------------+------

(pid=22428) [3,  2000] loss: 2.215
Result for DEFAULT_df0ef_00009:
  accuracy: 0.201
  date: 2021-07-28_00-42-14
  done: false
  experiment_id: 1c6056f30c4843f092492f6cb727925e
  hostname: DESKTOP-G26MRLK
  iterations_since_restore: 3
  loss: 2.125642303466797
  node_ip: 192.168.219.101
  pid: 22428
  should_checkpoint: true
  time_since_restore: 112.96740436553955
  time_this_iter_s: 36.170666456222534
  time_total_s: 112.96740436553955
  timestamp: 1627400534
  timesteps_since_restore: 0
  training_iteration: 3
  trial_id: df0ef_00009
  
== Status ==
Memory usage on this node: 5.3/15.9 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: -2.2590433113098145 | Iter 1.000: -2.302574312210083
Resources requested: 2.0/12 CPUs, 0/1 GPUs, 0.0/6.24 GiB heap, 0.0/3.12 GiB objects
Result logdir: C:\Users\PARK JIHO\ray_results\DEFAULT_2021-07-28_00-39-46
Number of trials: 10/10 (9 ERROR, 1 RUNNING)
+---------------------+----------+----------------

(pid=22428) [5,  2000] loss: 1.951
Result for DEFAULT_df0ef_00009:
  accuracy: 0.2848
  date: 2021-07-28_00-43-28
  done: false
  experiment_id: 1c6056f30c4843f092492f6cb727925e
  hostname: DESKTOP-G26MRLK
  iterations_since_restore: 5
  loss: 1.8587878770828248
  node_ip: 192.168.219.101
  pid: 22428
  should_checkpoint: true
  time_since_restore: 186.14989757537842
  time_this_iter_s: 36.549498558044434
  time_total_s: 186.14989757537842
  timestamp: 1627400608
  timesteps_since_restore: 0
  training_iteration: 5
  trial_id: df0ef_00009
  
== Status ==
Memory usage on this node: 5.8/15.9 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: -2.0200763750076294 | Iter 2.000: -2.2590433113098145 | Iter 1.000: -2.302574312210083
Resources requested: 2.0/12 CPUs, 0/1 GPUs, 0.0/6.24 GiB heap, 0.0/3.12 GiB objects
Result logdir: C:\Users\PARK JIHO\ray_results\DEFAULT_2021-07-28_00-39-46
Number of trials: 10/10 (9 ERROR, 1 RUNNING)
+---------------------+----------

2021-07-28 00:44:06,642	WARNING tune.py:507 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


KeyboardInterrupt: 